In [ ]:
#Ens connectem a la GPU
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [ ]:
#Lliberies utilitzades
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, Dense
from tensorflow.keras.layers import Input, UpSampling2D,BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import skimage.io as io
import numpy as np
from tqdm import tqdm
import nibabel as nib
import cv2
import numpy as np
from sklearn.model_selection import KFold
import pandas as pd
import scipy

In [ ]:
path = "HGG/HGG/"

In [ ]:
#Funció per llegir les imatges de la base de dades
def load_data(path,mida):
  my_dir = sorted(os.listdir(path))
  data = []
  gt = []
  seg_final=[]
  i=1
  for p in tqdm(my_dir): 
    if(p.startswith('Brats18')):
        data_list = sorted(os.listdir(path+p))
        seg = nib.load(path + p + '/'+ data_list[1]).get_fdata()
        
        flair = nib.load(path + p + '/'+ data_list[0]).get_fdata()
        
        t2 = nib.load(path + p + '/'+ data_list[4]).get_fdata()

        data.append([flair,t2])
        gt.append(seg)
        if i>=mida:
          break
        else:
          i=i+1
  data = np.asarray(data,dtype=np.float16)
  gt = np.asarray(gt,dtype=np.uint8)
  return data,gt

In [ ]:
nombre_mostres=285

In [ ]:
data1,gt1 = load_data(path,nombre_mostres)  

In [ ]:
#Modifiquem la forma dels vectors
data = np.transpose(data1,(0,4,2,3,1))
gt1=np.swapaxes(gt1,1,3)
gt1=np.swapaxes(gt1,2,3)
data = data.reshape([-1,240,240,2])
gt_final = gt1.reshape([-1,240,240,1])

In [ ]:
#Canviem els valors del vector gt_final
gt_final[np.where(gt_final!=0)]=1

In [ ]:
#Arquitectura de la U-NET
def unet():
    
    inputs = Input((240 , 240 , 2))
    
    conv1 = Conv2D(64, 3, activation='relu', padding='same') (inputs)
    batch1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, 3, activation='relu', padding='same') (batch1)
    batch1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2, 2))(batch1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same') (pool1)
    batch2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation='relu', padding='same') (batch2)
    batch2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2, 2))(batch2)
    
    conv3 = Conv2D(256, 3, activation='relu', padding='same') (pool2)
    batch3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation='relu', padding='same') (batch3)
    batch3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D((2, 2))(batch3)
    
    conv4 = Conv2D(512,3, activation='relu', padding='same') (pool3)
    batch4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, 3, activation='relu', padding='same') (batch4)
    batch4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D((2, 2))(batch4)
    
    conv5 = Conv2D(1024, 3, activation='relu', padding='same') (pool4)
    batch5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same') (batch5)
    batch5 = BatchNormalization()(conv5)
    
    up6 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same', activation='relu') (batch5)
    up6 = concatenate([up6, conv4])
    conv6 = Conv2D(512, 3, activation='relu', padding='same') (up6)
    batch6 = BatchNormalization()(conv6)
    conv6 = Conv2D(512,3, activation='relu', padding='same') (batch6)
    batch6 = BatchNormalization()(conv6)
    
    up7 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same', activation='relu') (batch6)
    up7 = concatenate([up7, conv3])
    conv7 = Conv2D(256,3, activation='relu', padding='same') (up7)
    batch7 = BatchNormalization(axis=1)(conv7)
    conv7 = Conv2D(256,3, activation='relu', padding='same') (batch7)
    batch7 = BatchNormalization()(conv7)
    
    up8 = Conv2DTranspose(128,2, strides=(2, 2), padding='same', activation='relu') (batch7)
    up8 = concatenate([up8, conv2])
    conv8 = Conv2D(128,3, activation='relu', padding='same') (up8)
    batch8 = BatchNormalization()(conv8)
    conv8 = Conv2D(128,3, activation='relu', padding='same') (batch8)
    batch8 = BatchNormalization(axis=1)(conv8)
    
    up9 = Conv2DTranspose(64,2, strides=(2, 2), padding='same') (batch8)
    up9 = concatenate([up9, conv1])
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same') (up9)
    batch9 = BatchNormalization()(conv9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same') (batch9)
    batch9 = BatchNormalization()(conv9)

    conv10 = Conv2D(1,1,activation='sigmoid')(batch9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy')

    return model

In [ ]:
#Dividim la base de dades en train i test
percentatge=abs(nombre_mostres*0.8)
#Train
input_train=data[0:int(percentatge*155),:,:,:]
target_train=gt_final[0:int(percentatge*155),:,:]
#Test
input_test=data[int(percentatge*155):nombre_mostres*155,:,:,:]
target_test=gt_final[int(percentatge*155):nombre_mostres*155,:,:,:]


In [ ]:
#Funcions que calculen el Dice Coefficient (DSC)
def dice_coef(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    union = np.sum(y_true_f) + np.sum(y_pred_f)
    if union==0: return 1
    intersection = np.sum(y_true_f * y_pred_f)
    return 2. * intersection / union

def mean_dice_coef(y_true, y_pred_bin,valor):
    # shape of y_true and y_pred_bin: (n_samples, height, width, n_channels)
    batch_size = y_true.shape[0]
    vector_dice=[]
    compt=1
    total_dice=[]
    for i in range(batch_size):
        if compt==155:
                vector_dice.append(dice_coef(y_true[i, :, :, valor], y_pred_bin[i, :, :,valor]))
                total_dice.append(np.mean(vector_dice))
                vector_dice=[]
                compt=1
        else:
                value=dice_coef(y_true[i, :, :, valor], y_pred_bin[i, :, :,valor])
                vector_dice.append(value)
                compt=compt+1
    return total_dice

In [ ]:
#Funció que calcula la distància de Hausdorff (HD)
def mean_distancia_hausdorff(y_true, y_pred_bin,valor):
    # shape of y_true and y_pred_bin: (n_samples, height, width, n_channels)
    batch_size = y_true.shape[0]
    vector_distancia=[]
    compt=1
    distancia=[]
    mean=[]
    total_distancia=[]
    for i in range(batch_size):
        if compt==155:
                distancia.append(scipy.spatial.distance.directed_hausdorff(y_true[i, :, :, valor],y_pred_bin[i, :, :,valor])[0])
                total_distancia.append(np.mean(distancia))
                distancia=[]
                compt=1
        else:
                distancia.append(scipy.spatial.distance.directed_hausdorff(y_true[i, :, :, valor],y_pred_bin[i, :, :,valor])[0])
                compt=compt+1
    return total_distancia

In [ ]:
num_folds=5
inputs = input_train
targets =target_train
kfold = KFold(n_splits=num_folds, shuffle=True,random_state=100)
fold_no=1

In [ ]:
#Entrenem els models
loss_per_fold=[]
vec_models=[]
scores_dice_total=[]
for train, test in kfold.split(inputs, targets):
    model = unet()

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(inputs[train], targets[train],batch_size=8,epochs=1)
    vec_models.append(model)

    #Predim el model
    prediccio=model.predict(inputs[test])
    dades=inputs[test]
    dades[np.where(dades>0)]=1
    prediccio=prediccio*dades
    prediccio=np.where(prediccio<0.5,0,1)
    
    #Calculem DSC del model corresponent
    score=mean_dice_coef(prediccio,targets[test],0)
    
    #Calculem loss del model corresponent
    scores_lose = model.evaluate(inputs[test], targets[test], verbose=0)
    
    #Mostrem els resultats
    scores_dice=np.mean(score)
    dice_coeficient="['dice_coef']"
    print(f'Score for fold {fold_no}: {model.metrics_names} of {scores_lose}%')
    print(f'Score for fold {fold_no}: {dice_coeficient} of {scores_dice}%')
    
    #Creem boxplot
    plt.boxplot(score)
    plt.show()
    scores_dice_total.append(score)
    score=[]
    prediccio=[]
    score_dice=0
    loss_per_fold.append(scores_lose)
    fold_no = fold_no + 1
    

In [ ]:
plt.boxplot(scores_dice_total)
plt.show()

In [ ]:
#vec_models[0].save('#1.1#SB_5_fold_50_epochs.h5')
#vec_models[1].save('#1.2#SB_5_fold_50_epochs.h5')
#vec_models[2].save('#1.3#SB_5_fold_50_epochs.h5')
#vec_models[3].save('#1.4#SB_5_fold_50_epochs.h5')
#vec_models[4].save('#1.5#SB_5_fold_50_epochs.h5')

In [ ]:
#Predim les imatges de test
Y_pre1=vec_models[0].predict(input_test)
Y_pre2=vec_models[1].predict(input_test)
Y_pre3=vec_models[2].predict(input_test)
Y_pre4=vec_models[3].predict(input_test)
Y_pre5=vec_models[4].predict(input_test)

# ENSEMBLE 1

In [ ]:
# Aquest mètode consisteix en comparar els valors de probabilitat dels píxels 
#de cada una de les 5 prediccions obtingudes, per tal de quedar-nos amb el valor amb la probabilitat més alta.

Y_pre_total_maxim=np.maximum(Y_pre1,Y_pre2)
Y_pre_total_maxim=np.maximum(Y_pre_total_maxim,Y_pre3)
Y_pre_total_maxim=np.maximum(Y_pre_total_maxim,Y_pre4)
Y_pre_total_maxim=np.maximum(Y_pre_total_maxim,Y_pre5)

Y_pre_total_maxim[np.where(Y_pre_total_maxim<0.5)]=0
Y_pre_total_maxim[np.where(Y_pre_total_maxim>=0.5)]=1

input_test=np.array(input_test,copy=True)
input_test2=input_test.copy()
input_test2[np.where(input_test2>0)]=1
Y_pre_total_maxim=Y_pre_total_maxim*input_test2

#Calculem accuracy
dades=mean_dice_coef(target_test, Y_pre_total_maxim,0)
print(f'Mitjana - DSC - ENSEMBLE 1: {np.mean(dades)}%')
print(f'Std - DSC - ENSEMBLE 1: {np.std(dades)}%')

In [ ]:
#Calculem distàcia de Hausdorff
dades_distancia_hausdorff=mean_distancia_hausdorff(target_test, Y_pre_total_maxim,0)
print(f'Mitjana - HD - ENSEMBLE 1:: {np.mean(dades_distancia_hausdorff)}')
print(f'Std - HD - ENSEMBLE 1: {np.std(dades_distancia_hausdorff)}')

In [ ]:
#Calculem la matriu de confusió
print('CONFUSION MATRIX ENSEMBLE 1:')
matriu=pd.crosstab(target_test[:,:,:,0].flatten(),Y_pre_total_maxim[:,:,:,0].flatten())
matrix=np.asarray(matriu)
tp=matrix[1,1]
tn=matrix[0,0]
fp=matrix[0,1]
fn=matrix[1,0]
matrix=[]
print(f'Sensibilitat ENSEMBLE 1 : {tp/(tp+fn)}')
print(f'Especificitat ENSEMBLE 1: {tn/(tn+fp)}')

# ENSEMBLE 2

In [ ]:
#Aquest mètode consisteix en calcular la mitjana dels diferents valors de 
#probabilitat dels píxels de cada una de les 5 prediccions obtingudes.

Y_pre_total_mitjana=Y_pre1+Y_pre2+Y_pre3+Y_pre4+Y_pre5
Y_pre_total_mitjana=Y_pre_total_mitjana/5

Y_pre_total_mitjana[np.where(Y_pre_total_mitjana<0.5)]=0
Y_pre_total_mitjana[np.where(Y_pre_total_mitjana>=0.5)]=1
Y_pre_total_mitjana=Y_pre_total_mitjana*input_test2

#Calculem accuracy
dades=mean_dice_coef(target_test, Y_pre_total_mitjana,0)
print(f'Mitjana - DSC - ENSEMBLE 2: {np.mean(dades)}%')
print(f'Std - DSC - ENSEMBLE 2: {np.std(dades)}%')

In [ ]:
#Calculem distància de Hausdorff
dades_distancia_hausdorff=mean_distancia_hausdorff(target_test, Y_pre_total_mitjana,0)
print(f'Mitjana - HD - ENSEMBLE 2:: {np.mean(dades_distancia_hausdorff)}')
print(f'Std - HD - ENSEMBLE 2: {np.std(dades_distancia_hausdorff)}')

In [ ]:
#Calculem la matriu de confusió
print('CONFUSION MATRIX ENSEMBLE 2:')
matriu=pd.crosstab(target_test[:,:,:,0].flatten(),Y_pre_total_mitjana[:,:,:,0].flatten())
matrix=np.asarray(matriu)
tp=matrix[1,1]
tn=matrix[0,0]
fp=matrix[0,1]
fn=matrix[1,0]
matrix=[]
print(f'Sensibilitat ENSEMBLE 2: {tp/(tp+fn)}')
print(f'Especificitat ENSEMBLE 2: {tn/(tn+fp)}')

# VOTING

In [ ]:
# Aquest mètode consisteix en observar els valors de probabilitat dels 
# píxels de cada una de les 5 prediccions obtingudes i, sumar quants valors 
# d'1 i quants valors de 0 hi ha. Un cop sumats, en la predicció final, posarem el
# valor (0 o 1) el qual té la suma més gran.

Y_pre1=Y_pre1*input_test2
Y_pre2=Y_pre2*input_test2
Y_pre3=Y_pre3*input_test2
Y_pre4=Y_pre4*input_test2
Y_pre5=Y_pre5*input_test2

Y_pre1=np.where(Y_pre1<0.5,0,1)
Y_pre2=np.where(Y_pre2<0.5,0,1)
Y_pre3=np.where(Y_pre3<0.5,0,1)
Y_pre4=np.where(Y_pre4<0.5,0,1)
Y_pre5=np.where(Y_pre5<0.5,0,1)

Y_pre_total_voting=Y_pre1+Y_pre2+Y_pre3+Y_pre4+Y_pre5
Y_pre_total_voting=np.where(Y_pre_total_voting>=3,1,0)
#Calculem accuracy
dades_voting=mean_dice_coef(target_test, Y_pre_total_voting,0)
print(f'Mitjana - DSC - VOTING: {np.mean(dades_voting)}')
print(f'Std - DSC - VOTING: {np.std(dades_voting)}')

In [ ]:
#Calculem distància de Hausdorff
dades_distancia_hausdorff=mean_distancia_hausdorff(target_test, Y_pre_total_voting,0)
print(f'Mitjana - HD - VOTING: {np.mean(dades_distancia_hausdorff)}')
print(f'Std - HD - VOTING: {np.std(dades_distancia_hausdorff)}')

In [ ]:
#Calculem la matriu de confusió
print('CONFUSION MATRIX VOTING:')
matriu=pd.crosstab(target_test[:,:,:,0].flatten(),Y_pre_total_voting[:,:,:,0].flatten())
matrix=np.asarray(matriu)
tp=matrix[1,1]
tn=matrix[0,0]
fp=matrix[0,1]
fn=matrix[1,0]
matrix=[]
print(f'Sensibilitat VOTING: {tp/(tp+fn)}')
print(f'Especificitat VOTING: {tn/(tn+fp)}')

# MIREM ELS RESULTATS D'UTILITZAR PER SEPARAT, ELS 5 MODELS OBTINGUTS EN L'ENTRENAMENT

In [ ]:
#Model 1
dades1=mean_dice_coef(target_test, Y_pre1,0)
print(f'Mitjana - DSC - MODEL 1: {np.mean(dades1)}')
print(f'std - DSC - MODEL 1: {np.std(dades1)}')
#Calculem distància de Hausdorff
dades_distancia_hausdorff=mean_distancia_hausdorff(target_test, Y_pre1,0)
print(f'Mitjana - HD - MODEL 1: {np.mean(dades_distancia_hausdorff)}')
print(f'Std - HD - MODEL 1: {np.std(dades_distancia_hausdorff)}')
dades_distancia_hausdorff=[]
#Calculem la matriu de confusió
print('CONFUSION MATRIX MODEL 1:')
matriu=pd.crosstab(target_test[:,:,:,0].flatten(),Y_pre1[:,:,:,0].flatten())
matrix=np.asarray(matriu)
tp=matrix[1,1]
tn=matrix[0,0]
fp=matrix[0,1]
fn=matrix[1,0]
matrix=[]
print(f'Sensibilitat MODEL 1: {tp/(tp+fn)}')
print(f'Especificitat MODEL 1: {tn/(tn+fp)}')
print(' ')


#Model 2
dades2=mean_dice_coef(target_test, Y_pre2,0)
print(f'Mitjana - DSC - MODEL 2: {np.mean(dades2)}')
print(f'std - DSC - MODEL 2: {np.std(dades1)}')
#Calculem distància de Hausdorff
dades_distancia_hausdorff=mean_distancia_hausdorff(target_test, Y_pre2,0)
print(f'Mitjana - HD - MODEL 2: {np.mean(dades_distancia_hausdorff)}')
print(f'Std - HD - MODEL 2: {np.std(dades_distancia_hausdorff)}')
dades_distancia_hausdorff=[]
#Calculem la matriu de confusió
print('CONFUSION MATRIX MODEL 2:')
matriu=pd.crosstab(target_test[:,:,:,0].flatten(),Y_pre2[:,:,:,0].flatten())
matrix=np.asarray(matriu)
tp=matrix[1,1]
tn=matrix[0,0]
fp=matrix[0,1]
fn=matrix[1,0]
matrix=[]
print(f'Sensibilitat MODEL 2: {tp/(tp+fn)}')
print(f'Especificitat MODEL 2: {tn/(tn+fp)}')
print(' ')

#Model 3
dades3=mean_dice_coef(target_test, Y_pre3,0)
print(f'Mitjana - DSC - MODEL 3: {np.mean(dades3)}')
print(f'std - DSC - MODEL 3: {np.std(dades3)}')
#Calculem distància de Hausdorff
dades_distancia_hausdorff=mean_distancia_hausdorff(target_test, Y_pre3,0)
print(f'Mitjana - HD - MODEL 3: {np.mean(dades_distancia_hausdorff)}')
print(f'Std - HD - MODEL 3: {np.std(dades_distancia_hausdorff)}')
dades_distancia_hausdorff=[]
#Calculem la matriu de confusió
print('CONFUSION MATRIX MODEL 3:')
matriu=pd.crosstab(target_test[:,:,:,0].flatten(),Y_pre3[:,:,:,0].flatten())
matrix=np.asarray(matriu)
tp=matrix[1,1]
tn=matrix[0,0]
fp=matrix[0,1]
fn=matrix[1,0]
matrix=[]
print(f'Sensibilitat MODEL 3: {tp/(tp+fn)}')
print(f'Especificitat MODEL 3: {tn/(tn+fp)}')
print(' ')


#Model 4
dades4=mean_dice_coef(target_test, Y_pre4,0)
print(f'Mitjana - DSC - MODEL 4: {np.mean(dades4)}')
print(f'std - DSC - MODEL 4: {np.std(dades4)}')
#Calculem distància de Hausdorff
dades_distancia_hausdorff=mean_distancia_hausdorff(target_test, Y_pre4,0)
print(f'Mitjana - HD - MODEL 4: {np.mean(dades_distancia_hausdorff)}')
print(f'Std - HD - MODEL 4: {np.std(dades_distancia_hausdorff)}')
dades_distancia_hausdorff=[]
#Calculem la matriu de confusió
print('CONFUSION MATRIX MODEL 4:')
matriu=pd.crosstab(target_test[:,:,:,0].flatten(),Y_pre4[:,:,:,0].flatten())
matrix=np.asarray(matriu)
tp=matrix[1,1]
tn=matrix[0,0]
fp=matrix[0,1]
fn=matrix[1,0]
matrix=[]
print(f'Sensibilitat MODEL 4: {tp/(tp+fn)}')
print(f'Especificitat MODEL 4: {tn/(tn+fp)}')
print(' ')

#Model 5
dades5=mean_dice_coef(target_test, Y_pre5,0)
print(f'Mitjana - DSC - MODEL 5: {np.mean(dades5)}')
print(f'std - DSDC - MODEL 5: {np.std(dades5)}')
#Calculem distància de Hausdorff
dades_distancia_hausdorff=mean_distancia_hausdorff(target_test, Y_pre5,0)
print(f'Mitjana - HD - MODEL 5: {np.mean(dades_distancia_hausdorff)}')
print(f'Std - HD - MODEL 5: {np.std(dades_distancia_hausdorff)}')
dades_distancia_hausdorff=[]
#Calculem la matriu de confusió
print('CONFUSION MATRIX MODEL 5:')
matriu=pd.crosstab(target_test[:,:,:,0].flatten(),Y_pre5[:,:,:,0].flatten())
matrix=np.asarray(matriu)
tp=matrix[1,1]
tn=matrix[0,0]
fp=matrix[0,1]
fn=matrix[1,0]
matrix=[]
print(f'Sensibilitat MODEL 5: {tp/(tp+fn)}')
print(f'Especificitat MODEL 5: {tn/(tn+fp)}')